In [160]:
pip install keras-tuner

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 1.7 MB/s eta 0:00:00


In [154]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.optimizers import Adam
import yfinance as yf
import datetime
from sklearn.preprocessing import MinMaxScaler

## Import data

In [127]:
start_date = datetime.datetime(2018, 1, 1)
end_date = datetime.datetime(2024, 1, 1)
btc_info = yf.Ticker("BTC-USD")

# pass the parameters as the taken dates for start and end
df = btc_info.history(start = start_date, end = end_date)

In [128]:
df.head()

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
2018-01-01 00:00:00+00:00,14112.200195,14112.200195,13154.700195,13657.200195,10291200000,0.0,0.0
2018-01-02 00:00:00+00:00,13625.000000,15444.599609,13163.599609,14982.099609,16846600192,0.0,0.0
2018-01-03 00:00:00+00:00,14978.200195,15572.799805,14844.500000,15201.000000,16871900160,0.0,0.0
2018-01-04 00:00:00+00:00,15270.700195,15739.700195,14522.200195,15599.200195,21783199744,0.0,0.0
2018-01-05 00:00:00+00:00,15477.200195,17705.199219,15202.799805,17429.500000,23840899072,0.0,0.0


In [129]:
df = df.drop(columns=['Dividends', 'Stock Splits'])

In [130]:
df.columns = ['open', 'high', 'low', 'close', 'vol']

In [131]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2191 entries, 2018-01-01 00:00:00+00:00 to 2023-12-31 00:00:00+00:00
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   open    2191 non-null   float64
 1   high    2191 non-null   float64
 2   low     2191 non-null   float64
 3   close   2191 non-null   float64
 4   vol     2191 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 102.7 KB


### Import new data

In [132]:
df_new = pd.read_csv('/content/BTC_new.csv')
df_new

,date,avg-block-size,avg-confirmation-time,blocks-size,cost-per-transaction-percent,cost-per-transaction,difficulty,estimated-transaction-volume-usd,estimated-transaction-volume,hash-rate,...,n-transactions-per-block,n-transactions-total,n-transactions,n-unique-addresses,output-volume,total-bitcoins,trade-volume,transaction-fees-usd,transaction-fees,transactions-per-second
0,2014-04-15,0.241621,NaN,17700.145907,4.312103,31.461496,6.119726e+09,6.737795e+07,146588.522890,5.932158e+04,...,400.049451,36853958.0,72809.0,156359.0,669588.742590,1.264937e+07,1.547115e+07,9.674245e+03,16.499545,NaN
1,2014-04-16,0.271066,NaN,17744.199359,2.176376,33.637923,6.119726e+09,1.235655e+08,236262.993009,4.532777e+04,...,431.408805,36926901.0,68594.0,149841.0,808972.686487,1.265370e+07,1.702180e+07,8.876057e+03,14.274446,NaN
2,2014-04-17,0.223713,NaN,17787.130813,3.960223,36.243881,6.978843e+09,6.833143e+07,129836.087704,6.036411e+04,...,385.739645,36995367.0,65190.0,131868.0,814063.297992,1.265771e+07,1.618354e+07,8.840666e+03,14.995538,NaN
3,2014-04-18,0.205560,NaN,17824.879464,6.280938,40.416473,6.978843e+09,4.413939e+07,88597.737449,5.411954e+04,...,356.225000,37060438.0,56996.0,128648.0,781666.572318,1.266182e+07,1.000750e+07,7.357991e+03,12.160113,NaN
4,2014-04-19,0.193319,NaN,17857.786874,7.890680,39.914189,6.978843e+09,3.141112e+07,65153.437509,5.654798e+04,...,336.572327,37117369.0,53515.0,118692.0,460095.566680,1.266583e+07,5.372537e+06,7.737646e+03,12.613149,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3645,2024-04-07,1.884625,41.076088,562982.133215,1.500297,142.915261,8.312700e+13,5.011808e+09,72211.397350,7.024851e+08,...,3094.882353,985002826.0,526130.0,652726.0,554943.049164,1.967582e+07,1.684330e+08,1.506788e+06,21.723003,2.850000
3646,2024-04-08,1.937860,44.144762,563302.582414,0.635671,159.310532,8.312700e+13,1.141858e+10,159660.840237,6.528979e+08,...,2883.651899,985528264.0,455617.0,685180.0,982666.187226,1.967685e+07,1.995999e+08,2.366374e+06,33.119632,3.633333
3647,2024-04-09,1.803056,71.778546,563608.974102,0.615871,148.461847,8.312700e+13,1.093957e+10,156080.237383,6.033108e+08,...,3108.294521,985984696.0,453811.0,683771.0,837041.011677,1.967780e+07,4.625718e+08,3.306938e+06,47.245769,2.900000
3648,2024-04-10,1.719400,84.627117,563872.274493,0.818675,132.023279,8.345099e+13,8.679207e+09,125497.148402,6.264041e+08,...,3516.913907,986437741.0,531054.0,703704.0,773781.907529,1.967873e+07,4.856081e+08,4.853314e+06,69.836212,4.650000


In [133]:
# drop the column with the null value
df_new = df_new.dropna(axis=1)

In [134]:
df_new.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3650 entries, 0 to 3649
Data columns (total 19 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   date                              3650 non-null   object 
 1   avg-block-size                    3650 non-null   float64
 2   blocks-size                       3650 non-null   float64
 3   cost-per-transaction              3650 non-null   float64
 4   difficulty                        3650 non-null   float64
 5   hash-rate                         3650 non-null   float64
 6   market-cap                        3650 non-null   float64
 7   market-price                      3650 non-null   float64
 8   median-confirmation-time          3650 non-null   float64
 9   miners-revenue                    3650 non-null   float64
 10  n-transactions-excluding-popular  3650 non-null   float64
 11  n-transactions-per-block          3650 non-null   float64
 12  n-tran

In [135]:
df_new['date'] = pd.to_datetime(df_new['date'])

<ipython-input-135-ab1b63ca2a3c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new['date'] = pd.to_datetime(df_new['date'])


### Merge dataset - Yahoo finance close column and the new dataset

In [136]:
df = df.reset_index()
df.columns = ['date', 'open', 'high', 'low', 'close', 'vol']
df.head()

,date,open,high,low,close,vol
0,2018-01-01 00:00:00+00:00,14112.200195,14112.200195,13154.700195,13657.200195,10291200000
1,2018-01-02 00:00:00+00:00,13625.000000,15444.599609,13163.599609,14982.099609,16846600192
2,2018-01-03 00:00:00+00:00,14978.200195,15572.799805,14844.500000,15201.000000,16871900160
3,2018-01-04 00:00:00+00:00,15270.700195,15739.700195,14522.200195,15599.200195,21783199744
4,2018-01-05 00:00:00+00:00,15477.200195,17705.199219,15202.799805,17429.500000,23840899072


In [137]:
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.strftime('%Y-%m-%d')
df['date'] = pd.to_datetime(df['date'])

In [138]:
df_final = df.merge(df_new, how='left', on='date')

In [139]:
df_final.isnull().sum()

date                                0
open                                0
high                                0
low                                 0
close                               0
vol                                 0
avg-block-size                      0
blocks-size                         0
cost-per-transaction                0
difficulty                          0
hash-rate                           0
market-cap                          0
market-price                        0
median-confirmation-time            0
miners-revenue                      0
n-transactions-excluding-popular    0
n-transactions-per-block            0
n-transactions-total                0
n-transactions                      0
output-volume                       0
total-bitcoins                      0
trade-volume                        0
transaction-fees-usd                0
transaction-fees                    0
dtype: int64

In [140]:
df_final = df_final.set_index('date')

In [141]:
# Delete the market-price
df_final = df_final.drop(columns=['market-price', 'vol'])

### Download the csv

In [171]:
df_final.to_csv('df_merged.csv')

In [142]:
df = df_final.copy()

In [143]:
# Prepare the volume and price differences, normalize volume
df_diff = df.diff().dropna()

In [144]:
df_aligned = df.loc[df_diff.index]

In [145]:
# Train data
# Period : From start of 2018 to end of 2022
mask_train = (df_diff.index >= "2018-01-01") & (df_diff.index < "2023-01-01")
df_train = df_diff.loc[mask_train].copy()
train_close = df_aligned.loc[mask_train, "close"].values
df_train["Relative_Close"] = train_close / train_close[0]

In [146]:
# Test data
# Period : Whole 2023
mask_test = (df_diff.index >= "2023-01-01") & (df_diff.index < "2024-01-01")  # December 2018 for testing
df_test = df_diff.loc[mask_test].copy()
test_close = df_aligned.loc[mask_test, "close"].values
df_test["Relative_Close"] = test_close / train_close[0]


In [147]:
df_train.head()

,open,high,low,close,avg-block-size,blocks-size,cost-per-transaction,difficulty,hash-rate,market-cap,...,n-transactions-excluding-popular,n-transactions-per-block,n-transactions-total,n-transactions,output-volume,total-bitcoins,trade-volume,transaction-fees-usd,transaction-fees,Relative_Close
date,,,,,,,,,,,,,,,,,,,,,
2018-01-02,-487.200195,1332.399414,8.899414,1324.899414,-0.002015,162.754373,-21.693162,8.555850e+09,1.439960e+06,7.953589e+09,...,92746.0,444.310729,241676.0,99223.0,853693.766232,2038.065956,-3.396831e+08,2.956526e+06,179.751940,1.000000
2018-01-03,1353.200195,128.200195,1680.900391,218.900391,0.024146,178.089352,-11.380017,0.000000e+00,-1.343962e+06,1.812910e+10,...,52008.0,528.022237,341182.0,54983.0,279946.016717,2050.177427,7.045421e+08,2.034675e+06,100.520496,1.014611
2018-01-04,292.500000,166.900391,-322.299805,398.200195,0.000319,167.274778,-1.935161,0.000000e+00,1.055970e+06,-2.301321e+09,...,26210.0,7.752199,396637.0,29045.0,-79059.151038,2033.974359,-3.053156e+08,2.276370e+05,28.575622,1.041189
2018-01-05,206.500000,1965.499023,680.599609,1830.299805,0.001803,179.046596,13.872140,0.000000e+00,-2.399933e+06,1.547523e+10,...,-80443.0,-133.257076,424746.0,-82301.0,183405.273086,1943.750000,2.516515e+08,-4.931253e+05,-76.905074,1.163355
2018-01-06,1984.899414,7.201172,1561.799805,97.500000,-0.013027,152.731365,15.360329,0.000000e+00,2.399933e+06,1.544869e+10,...,20479.0,-260.558400,342148.0,16140.0,58384.351269,1943.750000,3.182508e+08,5.126578e+05,-6.878115,1.169863


In [148]:
df_test.head()

,open,high,low,close,avg-block-size,blocks-size,cost-per-transaction,difficulty,hash-rate,market-cap,...,n-transactions-excluding-popular,n-transactions-per-block,n-transactions-total,n-transactions,output-volume,total-bitcoins,trade-volume,transaction-fees-usd,transaction-fees,Relative_Close
date,,,,,,,,,,,,,,,,,,,,,
2023-01-01,-55.759766,1.453125,3.714844,77.583984,-0.229010,167.280558,18.672292,0.000000e+00,-1.757957e+06,-2.516038e+08,...,-54721.0,-343.584739,243379.0,-54809.0,-69276.988571,971.875000,-1.775555e+07,-84204.371082,-5.067509,1.109663
2023-01-02,77.595703,128.904297,50.994141,63.390625,0.178520,130.593933,-14.536744,0.000000e+00,1.757957e+06,2.770948e+09,...,41739.0,260.392432,186842.0,41831.0,130051.645395,971.875000,-2.785593e+07,29655.898033,1.711537,1.113894
2023-01-03,63.337891,1.103516,50.142578,-8.613281,0.227036,159.245033,-12.013734,-1.154996e+12,-1.235785e+07,-2.887251e+07,...,42659.0,296.417832,228331.0,42708.0,139592.631805,967.994281,7.382737e+05,66539.799460,3.903536,1.113319
2023-01-04,-8.642578,204.138672,45.392578,183.380859,-0.023806,192.549611,-4.114519,-1.154996e+11,-1.952700e+07,2.502068e+09,...,5863.0,177.017483,273040.0,5881.0,46652.401504,930.289170,2.628487e+07,35809.759066,2.046453,1.125559
2023-01-05,183.267578,-80.564453,122.519531,-26.501953,-0.069425,174.724398,5.309258,0.000000e+00,2.372721e+07,-2.739302e+07,...,1464.0,-164.047882,276835.0,1455.0,7059.983108,933.712517,2.157660e+07,-7731.045214,-0.502997,1.123790


In [149]:
# Generate dataset function
def generate_dataset(df, seq_len):
    X_list, y_list = [], []
    for i in range(len(df) - seq_len):
        X_list.append(df.iloc[i:(i+seq_len), :].values)
        y_list.append(df["close"].iloc[i + seq_len])
    return np.array(X_list), np.array(y_list)

In [150]:
LAG = 1

In [151]:
# # Prepare training and test datasets
# X_train, y_train = generate_dataset(df_train, LAG)
# X_test, y_test = generate_dataset(pd.concat((df_train.iloc[-LAG:], df_test)), LAG)

In [152]:
validation_size = 0.2
n_validation = int(len(df_train) * validation_size)

df_val = df_train.iloc[-n_validation:]
df_train_reduced = df_train.iloc[:-n_validation]

X_train, y_train = generate_dataset(df_train_reduced, LAG)
X_val, y_val = generate_dataset(pd.concat((df_train_reduced.iloc[-LAG:], df_val)), LAG)
X_test, y_test = generate_dataset(pd.concat((df_train.iloc[-LAG:], df_test)), LAG)

In [155]:
num_samples, num_timesteps, num_features = X_train.shape
X_train_reshaped = X_train.reshape(-1, num_features)
scaler = MinMaxScaler(feature_range=(0, 1))
X_train_scaled = scaler.fit_transform(X_train_reshaped)
X_train_scaled = X_train_scaled.reshape(num_samples, num_timesteps, num_features)

In [156]:
num_samples_val, num_timesteps, num_features = X_val.shape
X_val_reshaped = X_val.reshape(-1, num_features)
X_val_scaled = scaler.transform(X_val_reshaped)
X_val_scaled = X_val_scaled.reshape(num_samples_val, num_timesteps, num_features)

In [157]:
num_samples_test, num_timesteps, num_features = X_test.shape
X_test_reshaped = X_test.reshape(-1, num_features)
X_test_scaled = scaler.transform(X_test_reshaped)
X_test_scaled = X_test_scaled.reshape(num_samples_test, num_timesteps, num_features)

### Hyperparameter Tuning

In [161]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch

<ipython-input-161-bf4685511596>:5: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [162]:
def build_model(hp):
    model = Sequential([
        LSTM(
            units=hp.Int('units1', min_value=32, max_value=256, step=32),
            return_sequences=True,
            input_shape=(X_train.shape[1], X_train.shape[2]),
            recurrent_dropout=hp.Float('recurrent_dropout1', min_value=0.0, max_value=0.5, step=0.1)
        ),
        Dropout(rate=hp.Float('dropout1', min_value=0.0, max_value=0.5, step=0.1)),
        LSTM(
            units=hp.Int('units2', min_value=32, max_value=256, step=32),
            recurrent_dropout=hp.Float('recurrent_dropout2', min_value=0.0, max_value=0.5, step=0.1)
        ),
        Dropout(rate=hp.Float('dropout2', min_value=0.0, max_value=0.5, step=0.1)),
        Dense(
            units=hp.Int('dense_units', min_value=16, max_value=128, step=16),
            activation='relu'
        ),
        Dense(1)
    ])

    model.compile(
        optimizer=Adam(
            learning_rate=hp.Float('learning_rate', min_value=1e-4, max_value=1e-2, sampling='LOG')
        ),
        loss='mse'
    )

    return model

In [163]:
tuner = RandomSearch(
    build_model,
    objective='val_loss',
    max_trials=10,  # Number of different configurations to try
    executions_per_trial=1,  # Number of models to train for each trial
    directory='my_dir',  # Directory to save logs and models
    project_name='lstm_tuning'
)

In [164]:
# Display search space summary
tuner.search_space_summary()

Search space summary
Default search space size: 8
units1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
recurrent_dropout1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dropout1 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
units2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
recurrent_dropout2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dropout2 (Float)
{'default': 0.0, 'conditions': [], 'min_value': 0.0, 'max_value': 0.5, 'step': 0.1, 'sampling': 'linear'}
dense_units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 128, 'step': 16, 'sampling': 'linear'}
learning_rate (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001

In [165]:
# Perform the hyperparameter search
tuner.search(
    X_train_scaled, y_train,
    epochs=5,
    validation_data=(X_val_scaled, y_val),
    callbacks=[tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)]
)

Trial 10 Complete [00h 00m 10s]
val_loss: 1029050.5

Best val_loss So Far: 1028692.25
Total elapsed time: 00h 01m 48s


In [166]:
# Initialize an empty list to hold each trial's data
trial_data = []

# Iterate through each trial and collect the data
for trial_id, trial in tuner.oracle.trials.items():
    if trial.status == "COMPLETED":
        # Extract the final validation loss for the trial
        val_loss = trial.metrics.get_best_value('val_loss')
        # Prepare a dictionary for the trial
        trial_info = {
            'Trial ID': trial_id,
            'MSE': val_loss
        }
        # Update the dictionary with the hyperparameters
        trial_info.update(trial.hyperparameters.values)
        # Append the dictionary to the list
        trial_data.append(trial_info)

# Convert the list of dictionaries to a DataFrame
df_trials = pd.DataFrame(trial_data)

# Display the DataFrame
df_trials

,Trial ID,MSE,units1,recurrent_dropout1,dropout1,units2,recurrent_dropout2,dropout2,dense_units,learning_rate
0,00,1.029163e+06,192,0.2,0.3,160,0.4,0.4,112,0.004985
1,01,1.028703e+06,128,0.2,0.3,160,0.2,0.1,16,0.000338
2,02,1.028693e+06,64,0.4,0.0,160,0.4,0.0,48,0.000140
3,03,1.028703e+06,32,0.0,0.0,96,0.1,0.1,16,0.000856
4,04,1.028692e+06,64,0.4,0.2,96,0.0,0.2,16,0.000205
5,05,1.028704e+06,256,0.4,0.4,224,0.3,0.0,80,0.000238
6,06,1.028875e+06,96,0.4,0.2,32,0.1,0.3,48,0.006524
7,07,1.028695e+06,224,0.0,0.3,96,0.1,0.2,64,0.000153
8,08,1.028694e+06,192,0.3,0.2,192,0.3,0.3,32,0.000135
9,09,1.029050e+06,32,0.3,0.2,224,0.0,0.1,16,0.006869


In [167]:
# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first LSTM layer is {best_hps.get('units1')},
the optimal dropout rates are {best_hps.get('dropout1')} for the first dropout layer and {best_hps.get('dropout2')} for the second dropout layer,
the optimal number of units in the second LSTM layer is {best_hps.get('units2')}, and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")


The hyperparameter search is complete. The optimal number of units in the first LSTM layer is 64,
the optimal dropout rates are 0.2 for the first dropout layer and 0.2 for the second dropout layer,
the optimal number of units in the second LSTM layer is 96, and the optimal learning rate for the optimizer
is 0.00020535161011692922.



In [168]:
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train_scaled, y_train, epochs=50, validation_data=(X_val_scaled, y_val))

Epoch 1/50
46/46 [==============================] - 7s 26ms/step - loss: 1011893.7500 - val_loss: 1028703.3125
Epoch 2/50
46/46 [==============================] - 0s 8ms/step - loss: 1011890.8750 - val_loss: 1028707.1250
Epoch 3/50
46/46 [==============================] - 0s 8ms/step - loss: 1011889.8125 - val_loss: 1028711.1875
Epoch 4/50
46/46 [==============================] - 0s 8ms/step - loss: 1011892.1250 - val_loss: 1028716.4375
Epoch 5/50
46/46 [==============================] - 0s 8ms/step - loss: 1011888.9375 - val_loss: 1028721.1250
Epoch 6/50
46/46 [==============================] - 0s 8ms/step - loss: 1011885.0000 - val_loss: 1028726.8750
Epoch 7/50
46/46 [==============================] - 0s 7ms/step - loss: 1011887.0000 - val_loss: 1028744.9375
Epoch 8/50
46/46 [==============================] - 0s 8ms/step - loss: 1011881.5000 - val_loss: 1028747.1250
Epoch 9/50
46/46 [==============================] - 0s 9ms/step - loss: 1011883.1875 - val_loss: 1028754.5625
Epoch 10/

In [169]:
predicted_prices = model.predict(X_test_scaled)

12/12 [==============================] - 1s 4ms/step


In [170]:
print("Test MSE:", np.mean((predicted_prices - y_test)**2))

Test MSE: 427964.97438321915


### Plot the comparison between actual and predicted value

In [ ]:
# y_close_test <- use value before minmax scaling
y_close_test = df_test_plot['close']
LSTM_pred = predicted_closing_prices.copy()

In [ ]:
import matplotlib.pyplot as plt

date_val = pd.to_datetime(y_close_test.index)
LSTM_close = y_close_test + (LSTM_pred - y_test)
fig = plt.figure(figsize=(13,8))
plt.plot(y_close_test, color='blue', linewidth=2, label='Actual')
plt.plot(LSTM_close, color='pink', linestyle='dashed',
linewidth=2, label="LSTM")
plt.title('Bitcoin Price Prediction', fontsize=20)
plt.xlabel('Date', fontsize=15)
plt.ylabel('Price', fontsize=15)
plt.legend()

In [ ]:
# Model architecture
tf.keras.utils.set_random_seed(4002)

model = Sequential([
    LSTM(50, activation='relu', input_shape=(X_train.shape[1], X_train.shape[2])),
    Dense(1)
])

model.compile(optimizer='adam', loss='mse')